In [1]:
import pandas as pd
import os
import tensorflow
import lightgbm

In [13]:
trackingData = pd.read_csv('cleanedTrackingData.csv')
playerData = pd.read_csv('playersDataClean.csv')
playData = pd.read_csv('plays.csv')


In [14]:
playData

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,...,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,...,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,...,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,...,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,2022100204,123,43293,Ezekiel Elliott,(13:31) E.Elliott right tackle to WAS 38 for 1...,1,1,10,DAL,WAS,...,0.638600,0.361400,-0.011542,0.011542,3.642571,-0.504018,NaN,NaN,NaN,NaN
12482,2022091200,3467,46189,Will Dissly,(6:08) G.Smith pass short right to W.Dissly to...,4,1,10,SEA,DEN,...,0.615241,0.384759,-0.025458,0.025458,1.434580,-0.444642,NaN,NaN,NaN,NaN
12483,2022101605,3371,44860,Joe Mixon,(9:35) (Shotgun) J.Mixon left end to CIN 47 fo...,4,1,10,CIN,NO,...,0.667054,0.332946,-0.005164,0.005164,2.115356,0.203819,NaN,NaN,NaN,NaN
12484,2022100207,2777,52449,Jonathan Taylor,(2:02) (Shotgun) J.Taylor up the middle to TEN...,3,1,10,IND,TEN,...,0.410611,0.589389,-0.046648,0.046648,3.946232,-0.976039,NaN,NaN,NaN,NaN


In [19]:
trackingData= trackingData.rename(columns={'nflId': 'ballCarrierId'})
trackingData.columns

Index(['gameId', 'playId', 'ballCarrierId', 'frameId', 'playDirection', 'x',
       'y', 'distanceToEndZone'],
      dtype='object')

In [22]:
# Assuming your DataFrames are named playData and trackingData

# Specify the columns to merge on
merge_columns = ['gameId', 'playId', 'ballCarrierId']

# Merge the DataFrames
merged_data = pd.merge(playData, trackingData, on=merge_columns, how='inner')

# Now merged_data contains the merged data based on the specified conditions


In [29]:
if merged_data['distanceToEndZone'].isna().any():
    print("Column 'column_name' contains NaN values.")
else:
    print("Column 'column_name' does not contain NaN values.")

Column 'column_name' does not contain NaN values.


In [30]:
playerData = playerData.rename(columns={'nflId': 'ballCarrierId'})


In [36]:
finalMergedData = pd.merge(merged_data,playerData,on=['ballCarrierId'],how='inner')

In [39]:
finalMergedData.columns

Index(['gameId', 'playId', 'ballCarrierId', 'ballCarrierDisplayName',
       'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'passLength',
       'penaltyYards', 'prePenaltyPlayResult', 'playResult',
       'playNullifiedByPenalty', 'absoluteYardlineNumber', 'offenseFormation',
       'defendersInTheBox', 'passProbability', 'preSnapHomeTeamWinProbability',
       'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded',
       'visitorTeamWinProbilityAdded', 'expectedPoints', 'expectedPointsAdded',
       'foulName1', 'foulName2', 'foulNFLId1', 'foulNFLId2', 'frameId',
       'playDirection', 'x', 'y', 'distanceToEndZone', 'Unnamed: 0',
       'height_x', 'weight_x', 'collegeName', 'Full Name', 'Team', 'Position',
       'yearsPro', 'overall_rating', 'teamId', 'age'],
      dtype='object')

In [40]:
finalMergedData.drop(inplace=True,columns=['Unnamed: 0'])

In [42]:
finalMergedData

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,height_x,weight_x,collegeName,Full Name,Team,Position,yearsPro,overall_rating,teamId,age
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,6-3,261,Iowa,Parker Hesse,Falcons,TE,2.0,52.0,14.0,27.0
1,2022091100,121,48723,Parker Hesse,(13:41) (Shotgun) M.Mariota pass short middle ...,1,3,4,ATL,NO,...,6-3,261,Iowa,Parker Hesse,Falcons,TE,2.0,52.0,14.0,27.0
2,2022091807,1217,48723,Parker Hesse,(5:30) M.Mariota pass short right to P.Hesse t...,2,4,1,ATL,LA,...,6-3,261,Iowa,Parker Hesse,Falcons,TE,2.0,52.0,14.0,27.0
3,2022100201,749,48723,Parker Hesse,(2:11) (Shotgun) M.Mariota pass deep left to P...,1,2,14,ATL,CLE,...,6-3,261,Iowa,Parker Hesse,Falcons,TE,2.0,52.0,14.0,27.0
4,2022103001,959,48723,Parker Hesse,(13:40) (Shotgun) M.Mariota pass short left to...,2,2,7,ATL,CAR,...,6-3,261,Iowa,Parker Hesse,Falcons,TE,2.0,52.0,14.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11343,2022101610,3088,48011,Tommy Sweeney,(2:05) J.Allen pass short right to T.Sweeney t...,3,2,6,BUF,KC,...,6-5,251,Boston College,Tommy Sweeney,Bills,TE,3.0,64.0,3.0,27.0
11344,2022100600,2612,48318,Devine Ozigbo,(8:51) D.Ozigbo up the middle to DEN 27 for 2 ...,3,1,10,DEN,IND,...,6-0,225,Nebraska,Devine Ozigbo,Saints,HB,3.0,63.0,27.0,25.0
11345,2022092600,1204,54553,Jalen Tolbert,(9:58) (Shotgun) C.Rush pass short right to J....,2,2,7,DAL,NYG,...,6-3,195,South Alabama,Jalen Tolbert,Cowboys,WR,0.0,71.0,11.0,23.0
11346,2022091803,2064,52620,Dezmon Patmon,(8:56) (Shotgun) M.Ryan pass short middle to D...,3,2,10,IND,JAX,...,6-4,220,Washington State,Dezmon Patmon,Colts,WR,2.0,64.0,10.0,24.0


In [44]:
finalMergedData.to_csv('finalMergedData.csv',index=False)